***SHEET 4***


CARLOS FAROUK ABDALÁ RINCÓN

TOMAS CANDEO MONTOYA

GABRIEL DAVID JIMÉNEZ MOSQUERA

In [2]:
#IMPORTS
import numpy as np
import random
import math
#import pandas as pd
#import matplotlib.pyplot as plt
#import requests as rq
#from itertools import combinations

### EXERCISE 2

#### EXERCISE 2.1

In [207]:
def kMeans(D,k,e,mu):
    t=0
    if mu==None:
        mu=[]
        min_Di = min(D[:, 0])
        max_Di = max(D[:, 0])
        min_Dj = min(D[:, 1])
        max_Dj = max(D[:, 1])
        for _ in range(k):
            centroid_i=random.uniform(min_Di,max_Di)
            centroid_j=random.uniform(min_Dj,max_Dj)
            mu.append([centroid_i,centroid_j])
    elif (len(mu)!=k):
        print("El número de centroides dado no coincide con las k's requeridas")
        return (None)
    e_calculated=e+1
    mu_previows= [[0,0]for _ in range(len(mu))]

    while(e_calculated>e):
        final_C=[]
        e_guide=0
        t+=1
        centroid_groups = [[]for _ in range(len(mu))]
        for _, point in enumerate(D): 
            difference_in_point=[]

            for _, centroid in enumerate(mu):
                difference_in_point.append(math.sqrt(np.linalg.norm(point - centroid)))
            nearest_index=np.argmin(difference_in_point)
            centroid_groups[nearest_index].append(point)
            final_C.append(nearest_index)

        for index, centroid_group in enumerate(centroid_groups):
            centroid_group_array=np.array(centroid_group)
            mean_centroid_group=np.mean(centroid_group_array,axis=0)
            mu[index]=mean_centroid_group

        for index, centroid_group_calculated in enumerate(mu):
            e_guide=e_guide+math.sqrt(np.linalg.norm(centroid_group_calculated - mu_previows[index]))
            e_calculated=e_guide
        mu_previows=mu.copy()
    return (final_C,mu)

        

D= np.array([
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8],
    [9, 10],
    [11, 12],
    [13, 14],
    [15, 16],
    [17, 18],
    [19, 20],
    [21, 22],
    [23, 24],
    [25, 26],
    [27, 28],
    [29, 30],
    [31, 32],
    [33, 34],
    [35, 36],
    [37, 38],
    [39, 40],
    [41, 42],
    [43, 44],
    [45, 46],
    [47, 48],
    [49, 50],
    [51, 52],
    [53, 54],
    [55, 56],
    [57, 58],
    [59, 60],
    [61, 62],
    [63, 64],
    [65, 66],
    [67, 68],
    [69, 70],
    [71, 72],
    [73, 74],
    [75, 76],
    [77, 78],
    [79, 80],
    [81, 82],
    [83, 84],
    [85, 86],
    [87, 88],
    [89, 90],
    [91, 92],
    [93, 94],
    [95, 96],
    [97, 98],
    [99, 100],
])

C,mu=kMeans(D,3,0.02,None)
print(C)
print(mu)

50
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[array([47., 48.]), array([15., 16.]), array([82., 83.])]
